# Aufgabe 2
## a
Lesen Sie die Trainingsdaten ein und teilen Sie sie in ein Validierungsdatenset (20%) und in ein eigentliches Trainigsdatenset (80%) auf. Finden Sie auf dem Validierungsdatensatz eine Regel für das Überleben alleine aufgrund der Klasse des Tickets (Pclass). Wenden Sie diese Regel auf die Validierungsdaten an. Wie gut ist die Genauigkeit (Anteil der korrekten Klassifikationen) auf den Validierungsdaten?

In [64]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/train.csv")
x = data[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]
y = data["Survived"]
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [65]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=42)

pclass_train_x = x_train["Pclass"].to_numpy()
pclass_train_y = y_train.to_numpy()
pclass_val_x = x_val["Pclass"].to_numpy()
pclass_val_y = y_val.to_numpy()

In [66]:
import random
def predict(x):
    random.seed(42)
    predicted = np.zeros(len(x),dtype=int)
    for i in range(len(x)):
        if x[i] == 1:
            predicted[i] = 1
        elif x[i] == 2:
            r = random.random()
            if r > 0.5:
                predicted[i] = 0
            else:
                predicted[i] = 1
        else:
            predicted[i] = 0
    return predicted

In [67]:
predicted = predict(pclass_val_x)
from sklearn.metrics import accuracy_score
accuracy_score(pclass_val_y,predicted)

0.6815642458100558

# b
Wenden Sie die Regel aus a) auf die Testdaten an und laden Sie Ihre Lösung hoch.

In [68]:
test_data = pd.read_csv("data/test.csv")
text_x = test_data["Pclass"].to_numpy()
predicted = predict(text_x)

# c
Trainieren Sie eine logistische Regression mit den Variablen 'Pclass'. Verwenden Sie die Klasse sklearn.linear_model.LogisticRegression. Berechnen Sie die Accuracy auf dem Validierungsset.


In [69]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
res = model.fit(pclass_train_x.reshape(len(pclass_train_x),1),y_train)
p = model.predict(pclass_val_x.reshape(len(pclass_val_x),1))
accuracy_score(p,pclass_val_y)

/home/onder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7039106145251397

# d
Verwenden Sie nun weitere Features. Die Variable Age enthält Missing values, die Sie durch folgenden code ersetzen können (was passiert da?)


In [70]:
final_train_x = x_train[["Pclass","Sex","Age"]]
final_train_y = y_train
final_train_x["Age"].fillna(final_train_x["Age"].median(skipna=True),inplace=True)

final_val_x = x_val[["Pclass","Sex","Age"]]
final_val_x["Age"].fillna(final_val_x["Age"].median(skipna=True),inplace=True)
final_train_x = pd.concat([final_train_x.drop('Sex', axis=1), pd.get_dummies(final_train_x['Sex'])], axis=1)
final_train_x = pd.concat([final_train_x.drop('Pclass', axis=1), pd.get_dummies(final_train_x['Pclass'])], axis=1)
final_val_x = pd.concat([final_val_x.drop('Sex', axis=1), pd.get_dummies(final_val_x['Sex'])], axis=1)
final_val_x = pd.concat([final_val_x.drop('Pclass', axis=1), pd.get_dummies(final_val_x['Pclass'])], axis=1)

/home/onder/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [71]:
final_train_x = final_train_x.to_numpy()
final_val_x = final_val_x.to_numpy()

model = LogisticRegression()
res = model.fit(final_train_x,y_train)
p = model.predict(final_val_x)
accuracy_score(p,pclass_val_y)

/home/onder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8044692737430168

# e
Weitere Klassifikatoren. Neben der logistischen Regression, gibt es weitere Klassifikatoren. Der Random-Forest ist ein recht stabiler Klassifikator, was wäre die Performance von diesem Klassifikator.

In [72]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
res = model.fit(final_train_x,y_train)
p = model.predict(final_val_x)
accuracy_score(p,pclass_val_y)

/home/onder/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7150837988826816

In [73]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
res = model.fit(final_train_x,y_train)
p = model.predict(final_val_x)
accuracy_score(p,pclass_val_y)

0.6983240223463687

In [74]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
res = model.fit(final_train_x,y_train)
p = model.predict(final_val_x)
accuracy_score(p,pclass_val_y)

0.7374301675977654

## Aufgabe 3
Mit den gleichen Daten, wie in der Aufgabe 2 d. Erstellen Sie ein fully connected neural network und fitten es an die Ttrainingsdaten. Verwenden Sie mindestens einen hidden Layer. Plotten Sie den Verlauf der Loss Kurve für die Trainings- und Validierungsdaten. Optional: Laden Sie Ihre beste Lösung auf Kaggle hoch.

In [100]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(20,10,5,3,2),max_iter=3000)
res = model.fit(final_train_x,y_train)
p = model.predict(final_val_x)
accuracy_score(p,y_val)

0.7374301675977654